# Working with GeLReportModels in the GMS

If programatically accessing cases in the Interpretation-API (aka CIP-API) you are encouraged to use the Gel Report Models application as it helps to map the JSON blobs from the API endpoints to Python objects which are easier and more efficient to work with. 

The Gel Report Models Application is available in PyPi here: https://pypi.org/project/GelReportModels/ thus can be installed with the following shell command:

`pip install gelreportmodels`

This will install the latest version of GeL Report Models (currently 7.3) 

If you are programmatically working with *GMS cases* in the Interpretation-API you need to use `v1.2.0` of the `participant` package of Gel Report Models which contains the `Referral` and `ReferralTest` data models

In [25]:
import os
import requests
from protocols.participant_1_2_0 import Referral, ReferralTest # these two models are GMS specific 
from protocols.reports_6_0_1 import InterpretationRequestRD, InterpretationDataRd, InterpretationDataCancer

In [26]:
host = "https://cipapi-gms-beta.gel.zone"

## Authentication

If you want to programatically query the GMS BETA Interpretation-API please contact Genomics England Service Desk - they will set you up in the application and issue you specific credentials for this

*NOTE THIS IS NOT FOR SYSTEM ACCESS TO THE Interpretation-API, FOR SYSTEM ACCESS YOUR ACTIVATE DIRECTORY SYSTEM TOKEN SHOULD BE USED TO MAKE QUERIES, SEE BELOW FOR INFO...*

### Example authentication using requests library and username and password

In [37]:
username = os.getenv('cipapi_user')
password = os.getenv('cipapi_pass')

In [38]:
def login(host, username, password):

    """ 
    Login to Interpretation-API and return a token 
    
    NOTE: it will not be possible to programmatically obtain a token using this method until your credentials 
    have first been manually registered in the application by Genomics England
    
    Please contact Genomics England Service Desk if you require this
    
    """

    url = '{}/api/2/get-token/'.format(host)
    token = requests.post(url, json={'username': username, 'password': password}).json().get('token')

    return token

In [29]:
token = login(host, username, password)

## Get an Interpretation Request (IR) for a case

In [30]:
interpretation_request_id = "88"
interpretation_request_version = "1"

### Example getting an IR using requests library

In [31]:
def get_ir(token, interpretation_request_id, interpretation_request_version, extra_params="?extra-params=show_referral"):
    
    """ Get the complete interpretation request data for a specific case and version 
    
    uses an extra parameter to get the referral information for GMS cases
    
    """
    
    url = '{host}/api/2/interpretation-request/{ir_id}/{ir_v}/{extras}'.format(host=host, 
                                                                               ir_id=interpretation_request_id, 
                                                                               ir_v=interpretation_request_version, 
                                                                               extras=extra_params)
    auth_header = {'Authorization': 'JWT {}'.format(token)}
    tv = requests.get(url, headers=auth_header).json()

    # check that the ir has the interpretation request data
    if 'interpretation_request_data' in tv and tv['interpretation_request_data']:
        return tv
    else:
        raise Exception('Case not found')

In [32]:
ir = get_ir(token=token, interpretation_request_id=interpretation_request_id, 
            interpretation_request_version=interpretation_request_version)

### The Interpretation Request JSON can be converted to a Python object using GeLReportModels

In [33]:
ir_model = InterpretationRequestRD.fromJsonDict(ir["interpretation_request_data"]["json_request"])

The pedigree associated with the case can then be acceesed like this: 

In [34]:
pedigree = ir_model.pedigree
type(pedigree)

protocols.reports_6_0_1.Pedigree

In [36]:
type(pedigree.members[0])

protocols.reports_6_0_1.PedigreeMember

### For GMS cases specific referral attributes from TOMs can be accessed like this

In [12]:
referral = Referral.fromJsonDict(ir['referral']['referral_data'])

In [13]:
referral.requester.organisationName

'Kettering General Hospital NHS Foundation Trust'

In [14]:
referral.referralTests[0].interpreter.organisationName

'Kettering General Hospital NHS Foundation Trust'

In [15]:
referral.referralTests[0].processingLab.organisationName

'Kettering General Hospital NHS Foundation Trust'

In [16]:
referral.clinicalIndication.clinicalIndicationFullName

'Hereditary ataxia with onset in adulthood'

In [17]:
referral.referralTests[0].referralTestOrderingDate.toJsonDict() # the date the test was ordered in TOMS

{'year': 2019, 'month': 6, 'day': 30}

# Examples using pycipapi python client

Pycipapi is a Python client developed by Genomics England to aid in interogating data in the Interpretation-API (aka CIP-API).
It is available in PyPi here: https://pypi.org/project/pycipapi/
therefore be installed like this: `pip install pycipapi`
doing so will also install the latest version of GeLReportModels  (also available in pypi: https://pypi.org/project/GelReportModels/)

The pycipapi client will automatically return JSON blobs as Gel Report Model objects

### Example authentication using pycipapi client: 

In [39]:
from pycipapi.cipapi_client import CipApiClient
cipapi = CipApiClient(host, user=username, password=password) # this creates a cip-api client which can then be used to make additional queries of the CIP-API data

### Example getting an interpretation request using pycipapi

In [40]:
case = cipapi.get_case(case_id=interpretation_request_id, 
                       case_version=interpretation_request_version,
                      **{'extra-params': 'show_referral'})

In [41]:
print(type(case.interpretation_request_payload))

<class 'protocols.reports_6_0_0.InterpretationRequestRD'>


In [42]:
print(type(case.pedigree))

<class 'protocols.reports_6_0_0.Pedigree'>


In [43]:
print(type(case.pedigree.members[0]))

<class 'protocols.reports_6_0_0.PedigreeMember'>


In [108]:
case.referral # TBC

AttributeError: 'CipApiCase' object has no attribute 'referral'

### listing referrals and associated interpretation requests

In [23]:

for referral in cipapi.list_referral():
    print('Referral id: {}'.format(referral.referral_id))
    for interpretation_request in referral.get_interpretation_requests(cipapi):
        print('\t* Interpretation Request {}: '.format(interpretation_request.case_id))

Referral id: r19552775693
	* Interpretation Request SAP-93-1: 
Referral id: r19754290876
	* Interpretation Request GEL-8-1: 
Referral id: r19448207345
	* Interpretation Request GEL-18-1: 
Referral id: r19156653850
	* Interpretation Request GEL-19-1: 
Referral id: r19337397961
	* Interpretation Request GEL-26-1: 
Referral id: r19698886184
	* Interpretation Request GEL-3-2: 
Referral id: r19392802654
	* Interpretation Request GEL-27-1: 
Referral id: r19628951458
	* Interpretation Request GEL-29-1: 
Referral id: r19087814411
	* Interpretation Request GEL-30-1: 
Referral id: r19475647240
	* Interpretation Request GEL-31-1: 
Referral id: r19239498445
	* Interpretation Request GEL-32-1: 
Referral id: r19281993271
	* Interpretation Request SAP-37-1: 
Referral id: r19016784391
Referral id: r19990439671
	* Interpretation Request SAP-37-2: 
Referral id: r19559016721
	* Interpretation Request SAP-62-1: 
Referral id: r19031314434
	* Interpretation Request SAP-63-1: 
Referral id: r19888729392
	* In

### Listing Interpretation requests and associated referrals

In [24]:
for interpretation_request in cipapi.get_cases(extra_params='show_referral', category='gms'):
    print(interpretation_request.case_id, interpretation_request.referral.referral_id)

GEL-8-1 r19754290876
GEL-18-1 r19448207345
GEL-19-1 r19156653850
GEL-26-1 r19337397961
GEL-3-2 r19698886184
GEL-27-1 r19392802654
GEL-29-1 r19628951458
GEL-30-1 r19087814411
GEL-31-1 r19475647240
GEL-32-1 r19239498445
SAP-37-1 r19281993271
SAP-37-2 r19990439671
SAP-62-1 r19559016721
SAP-63-1 r19031314434
SAP-64-1 r19888729392
SAP-66-1 r19804264624
SAP-68-1 r19066758142
SAP-69-1 r19879630291
SAP-70-1 r19935034984
SAP-71-1 r19252933191
SAP-57-2 r19585361336
SAP-58-3 r19407332690
SAP-59-2 r19503612036
SAP-60-2 r19498181097
SAP-62-2 r19806980097
SAP-63-2 r19226588580
SAP-64-2 r19643481497
SAP-67-2 r19206627604
SAP-68-2 r19115779204
SAP-70-2 r19182998460
SAP-71-2 r19238403159
ILMN-76-2 r19102201856
ILMN-83-1 r19518142077
ILMN-84-1 r19045844479
ILMN-85-1 r19865100258
ILMN-86-1 r19002254359
SAP-37-3 r19462737388
SAP-87-1 r19693455244
SAP-88-1 r19171183894
SAP-89-1 r19920504941
ILMN-90-1 r19809695566
ILMN-91-1 r19573546767
SAP-92-3 r19684356144
SAP-93-1 r19552775693
SAP-94-1 r19614421412


### Example Cancer Case
For GMS cancer cases it is recommened to get the cancer demographic and clinical data from the NGIS referral e.g. 

In [53]:
cancer_case = cipapi.get_case(case_id='86', 
                       case_version='1',
                      **{'extra-params': 'show_referral'})

In [55]:
cancer_case.case_id

'ILMN-86-1'

In [65]:
cancer_case.referral.referral_data.cancerParticipant.toJsonDict()

{'yearOfBirth': 1967,
 'morphology': None,
 'readyForAnalysis': True,
 'consentStatus': None,
 'testConsentStatus': None,
 'center': None,
 'individualId': None,
 'participantId': 'r19002254359',
 'primaryDiagnosisDisease': None,
 'primaryDiagnosisSubDisease': None,
 'sex': 'MALE',
 'additionalInformation': None,
 'assignedICD10': None,
 'tumourSamples': None,
 'germlineSamples': None,
 'versionControl': None,
 'participantUid': '53aea2ed-3114-4cfb-81ac-a546627c9330',
 'tumours': [{'tumourId': 'RP4_160003658',
   'tumourLocalId': 'RP4_160003658',
   'tumourType': 'SOLID_TUMOUR_PRIMARY',
   'tumourParentId': None,
   'tumourDiagnosisDate': {'year': 2019, 'month': 1, 'day': 31},
   'tumourDescription': None,
   'tumourMorphologies': [{'id': 'SCTID:271944004',
     'name': None,
     'value': None,
     'version': None}],
   'tumourTopographies': [{'id': 'SCTID:87784001',
     'name': None,
     'value': None,
     'version': None}],
   'tumourPrimaryTopographies': None,
   'tumourGrade':